In [68]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Flatten, Input
from gensim.models import Word2Vec

In [ ]:
!pip install gensim

In [ ]:
!pip install numpy --upgrade

In [3]:
!pip list

Package                                  Version        Editable project location
---------------------------------------- -------------- -------------------------
absl-py                                  1.4.0
accelerate                               0.34.2
aiobotocore                              2.15.1
aiofiles                                 22.1.0
aiohttp                                  3.9.5
aioitertools                             0.12.0
aiosignal                                1.3.1
aiosqlite                                0.20.0
albucore                                 0.0.17
albumentations                           1.4.17
alembic                                  1.13.3
altair                                   5.4.1
annotated-types                          0.7.0
annoy                                    1.17.3
ansicolors                               1.1.8
anyio                                    4.4.0
apache-beam                              2.46.0
appdirs                    

In [6]:
import gensim

In [7]:
from gensim.models import Word2Vec

In [8]:
jokes = pd.read_csv("../input/offense-classification-jokes/offensive_classification_jokes.csv")
jokes

,joke_type,joke,tokens
0,0,sherlock holmes and dr watson were going campi...,"['sherlock', 'holmes', 'and', 'dr', 'watson', ..."
1,0,"some cheesy ones, but they're some of my favor...","['some', 'cheesy', 'ones', ',', 'but', 'they',..."
2,0,what kind of horses go out after dark?nightmares!,"['what', 'kind', 'of', 'horses', 'go', 'out', ..."
3,0,a man in alaska was feeling lonely...so he dec...,"['a', 'man', 'in', 'alaska', 'was', 'feeling',..."
4,0,where did george washington keep his armies?in...,"['where', 'did', 'george', 'washington', 'keep..."
...,...,...,...
187883,0,"amazon is taking over the world, and googles j...","['amazon', 'is', 'taking', 'over', 'the', 'wor..."
187884,0,tifyou by mistakenly fornicating with a man at...,"['tifyou', 'by', 'mistakenly', 'fornicating', ..."
187885,0,"introducing ""all the children"" jokeshere in sw...","['introducing', '""', 'all', 'the', 'children',..."
187886,0,i successfully stole a case of toilet papergot...,"['i', 'successfully', 'stole', 'a', 'case', 'o..."


In [10]:
jokes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187888 entries, 0 to 187887
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   joke_type  187888 non-null  int64 
 1   joke       187887 non-null  object
 2   tokens     187888 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.3+ MB


In [14]:
jokes = jokes.dropna(subset=['joke'])
jokes = jokes.reset_index(drop=True)

In [16]:
jokes.sample(10)

,joke_type,joke,tokens
45724,1,why did the user create a new subredditbecause...,"['why', 'did', 'the', 'user', 'create', 'a', '..."
118864,0,a plane crashes next to a deserted island........,"['a', 'plane', 'crashes', 'next', 'to', 'a', '..."
161525,0,what do darth vader and christian grey have in...,"['what', 'do', 'darth', 'vader', 'and', 'chris..."
17089,1,i was surprised when i heard my wife screaming...,"['i', 'was', 'surprised', 'when', 'i', 'heard'..."
48771,1,i beat my kidsnot in fortnite in the fckin rea...,"['i', 'beat', 'my', 'kidsnot', 'in', 'fortnite..."
179186,0,"two peanuts walk in an alleyway,one was a salted.","['two', 'peanuts', 'walk', 'in', 'an', 'alleyw..."
147891,0,i'm putting sesame street characters in brine ...,"['i', ""'m"", 'putting', 'sesame', 'street', 'ch..."
115432,0,what's the difference between batman and a rob...,"['what', ""'s"", 'the', 'difference', 'between',..."
95415,1,many ’lost’ voters say they have found their c...,"['many', '’', 'lost', '’', 'voters', 'say', 't..."
18424,1,what does your niece and a microwave have in c...,"['what', 'does', 'your', 'niece', 'and', 'a', ..."


In [17]:
tokenized_jokes = jokes['tokens'].tolist()

In [18]:
w2v_model = Word2Vec(vector_size=100, window=5, sg=1, min_count=1, workers=4)

In [20]:
w2v_model.build_vocab(tokenized_jokes, progress_per=10000)

In [21]:
print("Vocabulary size:", len(w2v_model.wv.key_to_index))

Vocabulary size: 2604


In [23]:
w2v_model.train(tokenized_jokes, total_examples=w2v_model.corpus_count, epochs=w2v_model.epochs)

(105500805, 569193640)

In [25]:
def average_vector(tokens, model):
    # Initialize a list to hold the vectors
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    # If no vectors are found, return a zero vector
    if not vectors:
        return np.zeros(model.vector_size)
    # Calculate the average
    return np.mean(vectors, axis=0)

sz = len(jokes)
joke_vec = []
for i in range(sz):
    tokens = jokes.iloc[i]['tokens']  # Use .iloc to access the row by position
    joke_vec.append(average_vector(tokens, w2v_model))

    if i % 10000 == 0:
        print(f"Processed {i} jokes")

Processed 0 jokes
Processed 10000 jokes
Processed 20000 jokes
Processed 30000 jokes
Processed 40000 jokes
Processed 50000 jokes
Processed 60000 jokes
Processed 70000 jokes
Processed 80000 jokes
Processed 90000 jokes
Processed 100000 jokes
Processed 110000 jokes
Processed 120000 jokes
Processed 130000 jokes
Processed 140000 jokes
Processed 150000 jokes
Processed 160000 jokes
Processed 170000 jokes
Processed 180000 jokes


In [28]:
print(type(joke_vec))          # Should output: <class 'list'>
print(type(joke_vec[0]))       # Should output: <class 'numpy.ndarray'>
print(len(joke_vec))  

<class 'list'>
<class 'numpy.ndarray'>
187887


In [29]:
joke_vec_df = pd.DataFrame(joke_vec, columns=[f'feature_{i}' for i in range(1, 101)])
joke_vec_df

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100
0,-0.142229,0.118589,-0.075829,0.187504,0.040485,0.006685,-0.079875,0.201703,-0.082727,-0.201057,...,-0.030190,0.088362,0.093442,-0.234095,0.055956,-0.226748,0.077878,-0.038830,0.152219,-0.005158
1,-0.137749,0.123320,-0.078795,0.179578,0.044346,0.005416,-0.080833,0.201434,-0.078202,-0.207154,...,-0.034118,0.100942,0.100521,-0.227118,0.065440,-0.223017,0.088552,-0.028725,0.142152,-0.017605
2,-0.134027,0.120262,-0.069403,0.198497,0.034486,0.017157,-0.057995,0.214998,-0.079184,-0.211746,...,-0.044206,0.090938,0.102581,-0.237454,0.069898,-0.223032,0.063190,-0.034446,0.154980,0.007965
3,-0.137523,0.121525,-0.079695,0.182197,0.041420,0.007499,-0.083978,0.199667,-0.076615,-0.197903,...,-0.034185,0.089797,0.093300,-0.225826,0.060690,-0.221216,0.079669,-0.034710,0.149885,-0.016055
4,-0.137496,0.110624,-0.061254,0.201707,0.032061,0.007613,-0.063768,0.220763,-0.083319,-0.205729,...,-0.039887,0.101701,0.102250,-0.249277,0.073428,-0.221977,0.059713,-0.047906,0.173822,0.012980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187882,-0.140556,0.120501,-0.070569,0.187141,0.044632,0.003758,-0.085626,0.203533,-0.078454,-0.196224,...,-0.032381,0.088250,0.092344,-0.221852,0.063374,-0.226917,0.089472,-0.030066,0.141021,-0.009246
187883,-0.135135,0.114821,-0.066156,0.193695,0.041384,0.014408,-0.069579,0.211098,-0.080284,-0.195426,...,-0.038314,0.093956,0.092289,-0.235148,0.064970,-0.222356,0.063943,-0.045491,0.152638,0.013281
187884,-0.141261,0.118104,-0.074100,0.184140,0.041679,0.000664,-0.094177,0.203838,-0.081044,-0.189044,...,-0.026631,0.092936,0.094462,-0.211218,0.065445,-0.229946,0.094855,-0.039513,0.149966,-0.007825
187885,-0.133392,0.109879,-0.073904,0.196851,0.029211,0.010928,-0.065093,0.222921,-0.088549,-0.194210,...,-0.019374,0.092795,0.095472,-0.256311,0.061506,-0.218651,0.062660,-0.039239,0.155960,0.007575


In [30]:
jokes = pd.concat([jokes.reset_index(drop=True), joke_vec_df.reset_index(drop=True)], axis=1)
jokes

,joke_type,joke,tokens,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100
0,0,sherlock holmes and dr watson were going campi...,"['sherlock', 'holmes', 'and', 'dr', 'watson', ...",-0.142229,0.118589,-0.075829,0.187504,0.040485,0.006685,-0.079875,...,-0.030190,0.088362,0.093442,-0.234095,0.055956,-0.226748,0.077878,-0.038830,0.152219,-0.005158
1,0,"some cheesy ones, but they're some of my favor...","['some', 'cheesy', 'ones', ',', 'but', 'they',...",-0.137749,0.123320,-0.078795,0.179578,0.044346,0.005416,-0.080833,...,-0.034118,0.100942,0.100521,-0.227118,0.065440,-0.223017,0.088552,-0.028725,0.142152,-0.017605
2,0,what kind of horses go out after dark?nightmares!,"['what', 'kind', 'of', 'horses', 'go', 'out', ...",-0.134027,0.120262,-0.069403,0.198497,0.034486,0.017157,-0.057995,...,-0.044206,0.090938,0.102581,-0.237454,0.069898,-0.223032,0.063190,-0.034446,0.154980,0.007965
3,0,a man in alaska was feeling lonely...so he dec...,"['a', 'man', 'in', 'alaska', 'was', 'feeling',...",-0.137523,0.121525,-0.079695,0.182197,0.041420,0.007499,-0.083978,...,-0.034185,0.089797,0.093300,-0.225826,0.060690,-0.221216,0.079669,-0.034710,0.149885,-0.016055
4,0,where did george washington keep his armies?in...,"['where', 'did', 'george', 'washington', 'keep...",-0.137496,0.110624,-0.061254,0.201707,0.032061,0.007613,-0.063768,...,-0.039887,0.101701,0.102250,-0.249277,0.073428,-0.221977,0.059713,-0.047906,0.173822,0.012980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187882,0,"amazon is taking over the world, and googles j...","['amazon', 'is', 'taking', 'over', 'the', 'wor...",-0.140556,0.120501,-0.070569,0.187141,0.044632,0.003758,-0.085626,...,-0.032381,0.088250,0.092344,-0.221852,0.063374,-0.226917,0.089472,-0.030066,0.141021,-0.009246
187883,0,tifyou by mistakenly fornicating with a man at...,"['tifyou', 'by', 'mistakenly', 'fornicating', ...",-0.135135,0.114821,-0.066156,0.193695,0.041384,0.014408,-0.069579,...,-0.038314,0.093956,0.092289,-0.235148,0.064970,-0.222356,0.063943,-0.045491,0.152638,0.013281
187884,0,"introducing ""all the children"" jokeshere in sw...","['introducing', '""', 'all', 'the', 'children',...",-0.141261,0.118104,-0.074100,0.184140,0.041679,0.000664,-0.094177,...,-0.026631,0.092936,0.094462,-0.211218,0.065445,-0.229946,0.094855,-0.039513,0.149966,-0.007825
187885,0,i successfully stole a case of toilet papergot...,"['i', 'successfully', 'stole', 'a', 'case', 'o...",-0.133392,0.109879,-0.073904,0.196851,0.029211,0.010928,-0.065093,...,-0.019374,0.092795,0.095472,-0.256311,0.061506,-0.218651,0.062660,-0.039239,0.155960,0.007575


# Data modelling

In [48]:
X = jokes.drop(['joke_type', 'joke', 'tokens'], axis=1)
y = jokes.joke_type

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [50]:
X_train_reshaped = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

In [56]:
model = Sequential()
model.add(Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))  # Input layer
model.add(LSTM(128, return_sequences=False))  # LSTM layer with 128 units
model.add(Dropout(0.3))  # Dropout layer with 30% rate
model.add(Dense(1, activation='sigmoid'))

In [58]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 128)            │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,377 (458.50 KB)

 Trainable params: 117,377 (458.50 KB)

 Non-trainable params: 0 (0.00 B)

In [59]:
history = model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test), epochs=20, batch_size=64)
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

Epoch 1/20
2349/2349 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.6238 - loss: 0.6324 - val_accuracy: 0.6644 - val_loss: 0.5923
Epoch 2/20
2349/2349 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.6838 - loss: 0.5720 - val_accuracy: 0.6895 - val_loss: 0.5663
Epoch 3/20
2349/2349 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.6920 - loss: 0.5647 - val_accuracy: 0.6943 - val_loss: 0.5616
Epoch 4/20
2349/2349 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.6934 - loss: 0.5634 - val_accuracy: 0.6817 - val_loss: 0.5840
Epoch 5/20
2349/2349 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.6941 - loss: 0.5621 - val_accuracy: 0.6944 - val_loss: 0.5609
Epoch 6/20
2349/2349 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.6966 - loss: 0.5608 - val_accuracy: 0.6962 - val_loss: 0.5564
Epoch 7/20
2349/2349 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.6965 - loss: 0.5597 - val_accuracy: 0.6976 - val_loss: 0.5555
Epoch 8/20
2349/2349 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.6999 - loss: 0.5

In [60]:
predictions = model.predict(X_test_reshaped)

1175/1175 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [66]:
model.save('../working/offense_model.h5')

In [137]:
import pickle

# Save the model with pickle
with open('offense_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [67]:
w2v_model.save("../working/word2vec_model.model")

## Trying

In [69]:
w2v_model = Word2Vec.load("word2vec_model.model")

In [70]:
from tensorflow.keras.models import load_model

In [138]:
with open('offense_model.pkl', 'rb') as f:
    loaded_model2 = pickle.load(f)

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [71]:
loaded_model = load_model('offense_model.h5')

In [107]:
print(f"Vocabulary size: {len(w2v_model.wv.index_to_key)}")

Vocabulary size: 2604


In [108]:
tokens_from_joke = jokes.iloc[2]['tokens']
print("Tokens from joke:", tokens_from_joke)

# Check if all tokens exist in the model
missing_tokens = [token for token in tokens_from_joke if token not in w2v_model.wv]
print("Missing tokens:", missing_tokens)

Tokens from joke: ['what', 'kind', 'of', 'horses', 'go', 'out', 'after', 'dark?nightmares', '!']
Missing tokens: []


In [123]:
my_joke = "Yo mama is so ugly, she should have been a jew."

In [124]:
my_joke

'Yo mama is so ugly, she should have been a jew.'

In [125]:
slang_dict = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you",
    "ILU": "I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laughing My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don’t care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can’t stop laughing"
}
for slang, replacement in slang_dict.items():
    my_joke = my_joke.replace(slang, replacement)

In [126]:
my_joke = my_joke.lower()

In [97]:
import emoji
import re

In [127]:
def convert_emojis_and_remove_colons(text):
    text_with_emojis = emoji.demojize(text)
    return re.sub(r':(\w+):', r'\1', text_with_emojis)
my_joke = convert_emojis_and_remove_colons(my_joke)

In [99]:
import textblob as tb

In [128]:
blob = tb.TextBlob(my_joke)
my_joke = str(blob.correct())

In [101]:
import spacy as sp

In [129]:
nlp = sp.load("en_core_web_sm")
def tokenize_text(text):
    # Check if the input text is a string
    if isinstance(text, str):
        doc = nlp(text)  # Process the text with spaCy
        return [token.text for token in doc]
    else:
        return []  # Return an empty list for non-string inputs
tokens = tokenize_text(my_joke)

In [130]:
tokens

['to',
 'mamma',
 'is',
 'so',
 'ugly',
 ',',
 'she',
 'should',
 'have',
 'been',
 'a',
 'jew',
 '.']

In [131]:
def average_vector(tokens, model):
    # Initialize a list to hold the vectors
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    # If no vectors are found, return a zero vector
    if not vectors:
        return np.zeros(model.vector_size)
    # Calculate the average
    return np.mean(vectors, axis=0)

In [132]:
input = average_vector(tokens, w2v_model)

In [134]:
input

array([-7.80920610e-02,  9.28362310e-02, -3.01136244e-02,  1.84618577e-01,
       -1.32955266e-02,  5.20203495e-03, -1.32089943e-01,  2.74442583e-01,
       -1.16216026e-01, -2.34755203e-01, -1.13407774e-02, -8.98038149e-02,
       -1.44008085e-01,  2.36478284e-01, -4.49533314e-02,  1.10830581e-02,
        2.34112218e-01, -1.19111121e-01, -6.53830692e-02, -2.85397649e-01,
        4.41161506e-02, -7.99373016e-02,  2.56418496e-01, -1.20204985e-01,
        5.16466498e-02,  9.28679109e-03, -6.02765977e-02,  4.52158116e-02,
       -1.93850234e-01,  8.05642828e-02,  2.86282748e-01, -6.11477681e-02,
        4.03947607e-02,  2.40331795e-02,  9.51416045e-03,  7.26668313e-02,
       -4.67540771e-02, -4.43872623e-02, -3.61897759e-02, -4.26157899e-02,
        5.03869802e-02,  1.39408931e-01, -2.24821568e-01,  1.95195690e-01,
       -2.07724944e-02, -6.72517791e-02,  1.43778473e-01, -1.78123549e-01,
       -6.29164279e-02,  3.48844193e-02,  7.02435076e-02, -8.36394951e-02,
       -1.14257388e-01,  

In [133]:
input.shape

(100,)

In [135]:
input = input.reshape(-1, 1)  # Reshape to (100, 1)

In [140]:
input_reshaped = input.reshape(1, 1, input.shape[0])  # Reshape to (1, 1, 100)
predictions = loaded_model.predict(input_reshaped)
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.1457937]]


In [144]:
jokes[jokes.joke_type==1].joke[7303]

"what's the difference between a bag of cocaine and a four-year-old?eric clapton wouldn't let the cocaine fall out a window."